In [69]:
## import packages 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Homework 3:Eigenvector Decomposition

For this homework you will use the Communities and Crime Data Set

-  http://archive.ics.uci.edu/ml/datasets/communities+and+crime

- In the class we talked about how to reduce data dimensionality by extracting new set of featuresusing PCA, LDA and other methods.  The basis of these methods is the eigenvector decompositionof the data matrix.


- Load the crime dataset and store it as a matrix (The data is already normalized so you should not need to normalize the data further.)

- Report a table with the top 20 eigenvalues, is there a clear point where you could cut the dimensions?

In [71]:
## load the data in pandas
crime_data = pd.read_csv('communities.data.txt', sep=",", header=None)

In [72]:
crime_data.head

<bound method NDFrame.head of       0    1      2                       3    4     5     6     7     8    \
0       8    ?      ?            Lakewoodcity    1  0.19  0.33  0.02  0.90   
1      53    ?      ?             Tukwilacity    1  0.00  0.16  0.12  0.74   
2      24    ?      ?            Aberdeentown    1  0.00  0.42  0.49  0.56   
3      34    5  81440     Willingborotownship    1  0.04  0.77  1.00  0.08   
4      42   95   6096       Bethlehemtownship    1  0.01  0.55  0.02  0.95   
5       6    ?      ?       SouthPasadenacity    1  0.02  0.28  0.06  0.54   
6      44    7  41500             Lincolntown    1  0.01  0.39  0.00  0.98   
7       6    ?      ?               Selmacity    1  0.01  0.74  0.03  0.46   
8      21    ?      ?           Hendersoncity    1  0.03  0.34  0.20  0.84   
9      29    ?      ?             Claytoncity    1  0.01  0.40  0.06  0.87   
10      6    ?      ?            DalyCitycity    1  0.13  0.71  0.15  0.07   
11     36    ?      ?  RockvilleCe

In [3]:
print(crime_data.describe())

               0            4            5            6            7    \
count  1994.000000  1994.000000  1994.000000  1994.000000  1994.000000   
mean     28.683551     5.493982     0.057593     0.463395     0.179629   
std      16.397553     2.873694     0.126906     0.163717     0.253442   
min       1.000000     1.000000     0.000000     0.000000     0.000000   
25%      12.000000     3.000000     0.010000     0.350000     0.020000   
50%      34.000000     5.000000     0.020000     0.440000     0.060000   
75%      42.000000     8.000000     0.050000     0.540000     0.230000   
max      56.000000    10.000000     1.000000     1.000000     1.000000   

               8            9            10           11           12   \
count  1994.000000  1994.000000  1994.000000  1994.000000  1994.000000   
mean      0.753716     0.153681     0.144022     0.424218     0.493867   
std       0.244039     0.208877     0.232492     0.155196     0.143564   
min       0.000000     0.000000     0

#### Imputing the data
We can find that the data is not clean where there are lot of entries missing with '?' value. Hence we plan to imputing the missing data.

In [83]:
def isnumber(x):
    try:
        float(x)
        return True
    except:
        return False

## change all the non-number instances to nan
crime_data = crime_data[crime_data.applymap(isnumber)]

## convert as matrix and change the datatype to float
crime_matrix = crime_data.values.astype('float')

In [84]:
crime_matrix

array([[8.000e+00,       nan,       nan, ..., 3.200e-01, 1.400e-01,
        2.000e-01],
       [5.300e+01,       nan,       nan, ..., 0.000e+00,       nan,
        6.700e-01],
       [2.400e+01,       nan,       nan, ..., 0.000e+00,       nan,
        4.300e-01],
       ...,
       [9.000e+00, 9.000e+00, 8.007e+04, ..., 9.100e-01, 2.800e-01,
        2.300e-01],
       [2.500e+01, 1.700e+01, 7.260e+04, ..., 2.200e-01, 1.800e-01,
        1.900e-01],
       [6.000e+00,       nan,       nan, ..., 1.000e+00, 1.300e-01,
        4.800e-01]])

In [85]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')

In [86]:
imp_mean.fit(crime_matrix)

SimpleImputer(copy=True, fill_value=None, missing_values=nan, strategy='mean',
       verbose=0)

In [92]:
crime_data_matrix_p = imp_mean.transform(crime_matrix)

In [93]:
crime_data_matrix_p.shape

(1994, 127)

In [94]:
crime_data_matrix_p

array([[8.00000000e+00, 5.88268293e+01, 4.61883366e+04, ...,
        3.20000000e-01, 1.40000000e-01, 2.00000000e-01],
       [5.30000000e+01, 5.88268293e+01, 4.61883366e+04, ...,
        0.00000000e+00, 1.95078370e-01, 6.70000000e-01],
       [2.40000000e+01, 5.88268293e+01, 4.61883366e+04, ...,
        0.00000000e+00, 1.95078370e-01, 4.30000000e-01],
       ...,
       [9.00000000e+00, 9.00000000e+00, 8.00700000e+04, ...,
        9.10000000e-01, 2.80000000e-01, 2.30000000e-01],
       [2.50000000e+01, 1.70000000e+01, 7.26000000e+04, ...,
        2.20000000e-01, 1.80000000e-01, 1.90000000e-01],
       [6.00000000e+00, 5.88268293e+01, 4.61883366e+04, ...,
        1.00000000e+00, 1.30000000e-01, 4.80000000e-01]])

In [100]:
from numpy import linalg as LA

## Calculate the eigenvalues and eigenvectors

Instead of performing eigenvalue decomposition directly on the covariance matrix $R = A^TA$, I use the singular value decomposition on the crime data.

In this case, the singular values are the root square of the eigenvalue of covariance matrix and the $V^T$ contains the eigenvectors.

In [105]:
u, s, vt = np.linalg.svd(crime_data_matrix_p, full_matrices=True)

- The eigenvectors:

In [111]:
print("The eigenvectos of covariance matrix are:", vt)

The eigenvectos of covariance matrix are: [[-5.53123394e-04 -1.19968821e-03 -9.99999118e-01 ... -1.81501123e-06
  -3.75976360e-06 -4.52118901e-06]
 [-5.05036105e-02 -9.98714061e-01  1.22656965e-03 ... -2.77429497e-04
  -8.01371377e-05 -3.01512856e-04]
 [-9.96714151e-01  5.06316063e-02  4.98884478e-04 ...  2.71377291e-04
  -1.54157976e-03  1.79559232e-04]
 ...
 [ 1.25904828e-05 -1.26924180e-06 -6.69745864e-09 ... -1.28522827e-03
   2.50731577e-02  8.97904379e-04]
 [ 2.61194455e-06 -3.59124771e-07 -2.81955695e-09 ... -7.87703668e-04
   1.41537014e-03 -7.33994697e-04]
 [ 2.39320222e-07 -2.01118625e-08  4.04408331e-10 ...  1.89826373e-05
  -4.42752682e-05 -4.21173134e-05]]


- The eigenvalues:

In [109]:
squarer = lambda t: t ** 2
eigs = np.array([squarer(xi) for xi in s])

In [110]:
print("The eigenvalues are:", eigs)

The eigenvalues are: [4.77623528e+12 1.31476439e+07 6.86061468e+05 2.24667102e+04
 2.93218333e+03 2.00404191e+03 1.07771021e+03 6.52550070e+02
 5.52199932e+02 3.65353700e+02 3.12976961e+02 2.33596986e+02
 2.16949841e+02 1.74577407e+02 1.48709631e+02 1.12553687e+02
 1.08318653e+02 1.00416719e+02 8.79745696e+01 8.10405157e+01
 7.72919416e+01 6.61719946e+01 5.94737393e+01 5.63808567e+01
 5.33719911e+01 5.15343162e+01 5.00838642e+01 4.84287878e+01
 4.60808212e+01 4.31107356e+01 3.86129170e+01 3.80924776e+01
 3.54565214e+01 3.45851220e+01 3.42569669e+01 3.11674856e+01
 2.98114707e+01 2.92121567e+01 2.76841484e+01 2.63274978e+01
 2.59749720e+01 2.51015385e+01 2.23898797e+01 2.07589315e+01
 2.04167005e+01 1.98891681e+01 1.83075861e+01 1.80260674e+01
 1.73734393e+01 1.65075319e+01 1.60976876e+01 1.48762702e+01
 1.45953166e+01 1.39683381e+01 1.32286103e+01 1.22747031e+01
 1.19707897e+01 1.12052260e+01 1.07482695e+01 1.00432677e+01
 9.25450278e+00 8.84342317e+00 8.46459325e+00 8.24704376e+00
 7.

In [112]:
## Check on the shape of the singular values 
## to confirm that there is no zero eigenvalues
eigs.shape

(127,)

### List the top 20 eigenvalues

In [113]:
print("The top 20 eigenvalues are \n", eigs[0:20])

The top 20 eigenvalues are 
 [4.77623528e+12 1.31476439e+07 6.86061468e+05 2.24667102e+04
 2.93218333e+03 2.00404191e+03 1.07771021e+03 6.52550070e+02
 5.52199932e+02 3.65353700e+02 3.12976961e+02 2.33596986e+02
 2.16949841e+02 1.74577407e+02 1.48709631e+02 1.12553687e+02
 1.08318653e+02 1.00416719e+02 8.79745696e+01 8.10405157e+01]


There is no clear point to cut off the dimension, since there is no estimation of error based just on the how much dimension reduced. This is still an open question for Model Order Reduction. In the end, we need an evaluation function to tell whether the reduced model is enough. 

Typically we could test the reduced dimension by adding up the dimension slowly.

For example, we can choose first 5~10 eigenvalues to decide whether it is enough for the dimension reduction. After that, we can test whether the reduced model satisfy the degree of accuracy we needed by adding the dimension step by step. 